 1. 这个需要你先用历史波动率c2c模型分别计算出不同到期日的atm的波动率，
 2. 之后再根据wing模型，具体参数你可以做一个参数组，分为微笑（曲率：小中大），右偏（曲率：小中大）：小中大，左偏（曲率：小中大），计算出一个波动率曲面，之后把上面那两个表自动生成出来。

- 另外，行权价距离atm值15%的位置的波动率是atm波动率的30%，6%的位置波动率是atm波动率的16%，这样就能生成5个点，你用这5点拟合一下图形，把剩下的生成出来就行了

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from WindPy import w
w.start()

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2017 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


.ErrorCode=0
.Data=[OK!]

In [2]:
def get_stock_data(code_dir,period, start):
    pre_start=w.tdaysoffset(-np.max(period), start, "Period=D").Data[0][0]
    data = w.wsd(code_dir, "pre_close,high,low,close,lastradeday_s", pre_start, start)
    df = pd.DataFrame(data.Data, index=data.Fields).T
    df['DailyReturn'] = df['CLOSE'] / df['PRE_CLOSE']    
    df['LASTRADEDAY_S'] = pd.to_datetime(df['LASTRADEDAY_S'])
    df.set_index('LASTRADEDAY_S', inplace=True)
    return df   

In [3]:
def calculate_vol(data, start,period, method='CLC'):
    ## ewma
    if method == 'EWMA':
        his_vol = [((np.log(data['DailyReturn'].astype('float32'))**2).ewm(span=day).mean())[-1]**0.5*252**0.5 for day in period ]
    
    ## clc
    elif method == 'CLC': 
        his_vol = np.array([(np.log(data['DailyReturn'].astype('float32')).rolling(window=day).std())[-1]*252**0.5  for day in period])
    
    ## Parkinson
    elif method == 'PARKINSON': 
        his_vol = [(data['HIGH']/data['LOW']**2/(4*np.log(2))).rolling(window=day).mean()[-1]*252**0.5 for day in period]

    return his_vol

In [15]:
def fit_function(X, A, boundry=10):
    I1 = (X>=0) & (X<=boundry)
    I2 = (X<=0) & (X>=-boundry)
    y = A[:, 0:1] + A[:,1:2]*X + A[:, 2:3]*X**2*I1 + A[:,3:4]*X**2*I2
    return y

In [29]:
def gradient_descend_constrain(x, y, I1, I2, alpha=0.01, n=10000, theta=1):
    
    m = x.shape[1]
    A = np.random.randn(x.shape[0],4)
    costs = []
    for i in range(n):
        hypothesis = fit_function(x, A)
        loss = hypothesis - y
        cost = np.sum(loss ** 2) / (2 *m)
        costs.append(cost)

        a3, a4 = A[:, 2], A[:, 3] 

        a1_grad = np.sum(loss, axis=1) / m
        a2_grad = np.sum(x*loss, axis=1) / m
        a3_grad = np.sum(x**2*loss*I1, axis=1) / m  + theta*a3*(a3<0)
        a4_grad = np.sum(x**2*loss*I2, axis=1) / m  + theta*a4*(a4<0)

        A[:, 0] -=  alpha * a1_grad
        A[:, 1] -=  alpha * a2_grad
        A[:, 2] -=  alpha * a3_grad
        A[:, 3] -=  alpha * a4_grad
    return A, costs

In [41]:
def output_iv(file_dir, code_dir, start=None, output_dir=None):
    start = start if start else str(pd.datetime.today().date())
    ## basic info
    basic_data = pd.read_excel(file_dir)
    columns = basic_data.columns
    index = basic_data.loc[basic_data['C/P']=='C'].index 
    period = (pd.to_datetime(index.astype('str')) - pd.datetime.today()).days.values
    predict_k = columns[2:].astype('float16').values

    ## 得到历史波动率
    df = get_stock_data(code_dir, period, start)
    ATM = df['CLOSE'][-1]
    his_vol = calculate_vol(df, start, period)

    ## 这里可以修改参数
    Vol_data = pd.DataFrame([his_vol*(1+0.3), his_vol*(1+0.16), his_vol, his_vol*(1+0.16), his_vol*(1+0.3)],
                 index=[ATM*(1-0.15), ATM*(1-0.06), ATM, ATM*(1+0.06), ATM*(1+0.15) ], columns=index).T

    x_ori = np.log(Vol_data.columns.values / ATM)
    x_ori = np.repeat(x_ori.reshape((1,-1)), Vol_data.shape[0], axis=0)
    y_ori = Vol_data.values

    x_std = np.std(x_ori)
    y_std = np.std(y_ori)

    x = x_ori / x_std
    y = y_ori / y_std

    boundry = 10
    I1 = (x>=0) & (x<=boundry)
    I2 = (x<=0) & (x>=-boundry)

    ## get_parameter
    A, _ = gradient_descend_constrain(x, y, I1, I2)

    ## get_result
    result_data = basic_data.copy()
    result = [y_std*fit_function(np.log(predict_k/ATM)/x_std, A[i:i+1,:]).reshape(-1)*100 for i in range(A.shape[0])]
    result_data.iloc[::2, 2:] = result
    result_data.iloc[1::2,2:] = result

    ## output 
    output_dir = output_dir if output_dir else code_dir+'.xls'
    result_data.to_excel(output_dir)
    
    return result_data

In [42]:
file_dir = './../../data/300070.xls'
code_dir = '300070.SZ'


output = output_iv(file_dir, code_dir)